# ACIS Data Download

## 1. Data pull from LOCA dataset (Gridded Data 2)
## (Historical as well as Projected RCP85)

In [2]:
import json
import pandas as pd
import numpy as np
import requests
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

In [11]:
elements = [{"name":"maxt","interval":"mly","duration":"mly","reduce":"mean","area_reduce":"county_mean"},
            {"name":"avgt","interval":"mly","duration":"mly","reduce":"mean","area_reduce":"county_mean"},
            {"name":"mint","interval":"mly","duration":"mly","reduce":"mean","area_reduce":"county_mean"},
            {"name":"pcpn","interval":"mly","duration":"mly","reduce":"sum","area_reduce":"county_mean"}]

states = 'AL'
# states = 'AL,AK,AZ,AR,CA,CO,CT,DE,DC,FL,GA,HI,ID,IL,IN,IA,KS,KY,LA,ME,MT,NE,NV,NH,NJ,NM,NY,NC,ND,OH,OK,OR,MD,MA,MI,MN,MS,MO,PA,RI,SC,SD,TN,TX,UT,VT,VA,WA,WV,WI,WY'

# elements =  [{"name":"pcpn","interval":"mly","duration":"mly","reduce":"sum","area_reduce":"county_mean"}]

# elements = [{"name":"mint","interval":"mly","duration":"mly","reduce":"mean","area_reduce":"county_mean"}]

# {"state":"AL,AK","sdate":"2024-06","edate":"2024-08","grid":"loca:wmean:rcp85","elems":[{"name":"mint","interval":"mly","duration":"mly","reduce":"mean","area_reduce":"county_mean"}]}

In [12]:
%%time

# Set parameters
start = "1970-01"
end = "2050-12"

# Set base URL, and pull the json data down using requests
base_url = 'http://grid2.rcc-acis.org/GridData' 
input_dict = {"state": states,"grid":"loca:wmean:rcp85",
              "sdate": start, "edate": end, 
              "elems": elements}

#Keep trying till success
while True:
    try:
        with requests.post(base_url, json=input_dict) as response:
            rawjson = response.content
    except:
        print('failed')
        continue
    else:
        break

# Convert raw json to a Python variable
newdata = json.loads(rawjson)['data']

CPU times: total: 234 ms
Wall time: 881 ms


In [13]:
# 81 years of data, we expect 81*12 months
print(81*12)

# How many months in the downloaded data?
len(newdata)

972


972

In [14]:
# How many counties in each month?
len(newdata[0][1])

90

In [15]:
# View the month
[(newdata[20][0])]

['1971-09']

In [16]:
# Trial, don't use

# mint = pd.DataFrame(newdata[20][1], index = ['mint']).transpose()
# maxt = pd.DataFrame(newdata[20][2], index = ['maxt']).transpose()
# avgt = pd.DataFrame(newdata[20][3], index = ['avgt']).transpose()
# pcpn = pd.DataFrame(newdata[20][4], index = ['pcpn']).transpose()
# combined_onemonth = pd.concat([mint,maxt,avgt,pcpn], axis=1)
# combined_onemonth.insert(0,'month',newdata[20][0])
# combined_onemonth = combined_onemonth.reset_index().rename(columns = {'index':'county'})
# combined_onemonth

In [18]:
%%time
# Loop to convert the JSON data into a dataframe


final = pd.DataFrame(columns=['county', 'month', 'mint', 'maxt', 'avgt', 'pcpn'])

for i in range(len(newdata)):
    mint = pd.DataFrame(newdata[i][1], index = ['mint']).transpose()
    maxt = pd.DataFrame(newdata[i][2], index = ['maxt']).transpose()
    avgt = pd.DataFrame(newdata[i][3], index = ['avgt']).transpose()
    pcpn = pd.DataFrame(newdata[i][4], index = ['pcpn']).transpose()
    combined_onemonth = pd.concat([mint,maxt,avgt,pcpn], axis=1)
    combined_onemonth.insert(0,'month',newdata[i][0])
    combined_onemonth = combined_onemonth.reset_index().rename(columns = {'index':'county'})
    combined_onemonth
    final = pd.concat([final, combined_onemonth])

CPU times: total: 4.72 s
Wall time: 4.69 s


In [19]:
final

,county,month,mint,maxt,avgt,pcpn
0,01001,1970-01,55.172170,44.383316,33.599150,4.473824
1,01003,1970-01,60.202305,49.611065,39.017280,5.020435
2,01005,1970-01,57.605236,46.414818,35.228480,4.282023
3,01007,1970-01,53.705177,42.364760,31.027027,4.858382
4,01009,1970-01,50.246710,39.333240,28.421835,5.060015
...,...,...,...,...,...,...
85,13285,2050-12,62.295643,49.919212,37.548447,4.905094
86,13295,2050-12,56.037693,44.593723,33.156017,5.410181
87,13313,2050-12,56.542984,45.611076,34.676410,5.219688
88,28057,2050-12,59.133934,47.982952,36.826977,5.548898


## 2. Download historical data (MultiStn data)
#### https://www.rcc-acis.org/docs_webservices.html

In [20]:
# Usual library imports
import json
import pandas as pd
import numpy as np
import requests

# Set some parameters
start = "2018-08-01"
end = "2019-08-31"
county = ["36119", "36001"]

# Set base URL, and pull the json data down using requests
base_url = "http://data.rcc-acis.org/MultiStnData"
input_dict = {"county": county, "meta": [], 
              "sdate": start, "edate": end, 
              "elems": "mly_mean_avgt,mly_min_mint,mly_max_maxt,mly_sum_hdd,mly_sum_gdd,mly_sum_pcpn,mly_sum_snow"}
with requests.post(base_url, json=input_dict) as response:
    rawjson = response.content

# Convert raw json to a Python variable
newdata = json.loads(rawjson)['data']

In [21]:
newdata

[{'meta': {'county': '36119', 'uid': 18415},
  'data': [['74.66', '54', '93', '0', '773', '8.52', '0.0'],
   ['66.78', '45', '93', '70', '512', '5.73', '0.0'],
   ['52.90', '29', '81', '387', '191', '5.01', '0.0'],
   ['38.73', '7', '69', '780', '22', '8.80', '8.0'],
   ['34.32', '15', '61', '940', '6', '5.16', '0.5'],
   ['28.24', '-0', '55', '1131', '0', '6.57', '2.6'],
   ['30.54', '-3', '64', '956', '0', '3.15', '4.0'],
   ['35.34', '7', '72', '911', '11', '3.15', '11.0'],
   ['50.38', '22', '78', '417', '104', '5.58', '0.0'],
   ['58.03', '39', '86', '224', '278', '5.78', '0.0'],
   ['66.33', '40', '88', '41', '499', '4.67', '0.0'],
   ['75.23', '57', '96', '0', '791', '3.25', '0.0'],
   ['71.61', '51', '92', '3', '676', '2.27', '0.0']]},
 {'meta': {'county': '36001', 'uid': 18747},
  'data': [['75.13', '57', '96', '0', '787', '4.20', '0.0'],
   ['67.20', '43', '94', '88', '524', '5.62', '0.0'],
   ['50.74', '27', '83', '454', '135', '3.77', 'T'],
   ['36.97', '4', '64', '834', '1

In [22]:
# Here we convert the newdata variable to a Pandas dataframe

# Set some more parameters that will be reused
dates = pd.date_range(start=start,end=end, freq='M')
columns = ["mly_mean_avgt", "mly_min_mint", "mly_max_maxt", "mly_sum_hdd", "mly_sum_gdd", "mly_sum_pcpn", "mly_sum_snow"]

# With some pandas gymnastics, loop through the json data and put it in a dataframe
appended_data = []

for i in range(len(newdata)):
    data = pd.DataFrame(newdata[i]['data'], index=dates, columns=columns)
    data.insert(0,"county", newdata[i]['meta']['county'] )
    data.insert(1,"uid", newdata[2]['meta']['uid'] )
    appended_data.append(data)
    
appended_data = pd.concat(appended_data)

In [23]:
appended_data

,county,uid,mly_mean_avgt,mly_min_mint,mly_max_maxt,mly_sum_hdd,mly_sum_gdd,mly_sum_pcpn,mly_sum_snow
2018-08-31,36119,18660,74.66,54,93,0,773,8.52,0.0
2018-09-30,36119,18660,66.78,45,93,70,512,5.73,0.0
2018-10-31,36119,18660,52.90,29,81,387,191,5.01,0.0
2018-11-30,36119,18660,38.73,7,69,780,22,8.80,8.0
2018-12-31,36119,18660,34.32,15,61,940,6,5.16,0.5
...,...,...,...,...,...,...,...,...,...
2019-04-30,36119,18660,M,M,M,M,M,4.25,M
2019-05-31,36119,18660,M,M,M,M,M,8.06,M
2019-06-30,36119,18660,M,M,M,M,M,3.03,M
2019-07-31,36119,18660,M,M,M,M,M,5.60,M
